# Efficient Gaussian Random Field Inference on Graphs - Regression Task on Traffic Dataset

This notebook contains the following:

1. Showing the baseline performance using the exact diffusion kernel.
2. Showing the similar level performance using Product of Feature Matrices (PoFM) kernel, compared across the parameter 'max_expansion' which determines the order of the approximation.
3. Showing the convergence of the GRF to the PoFM kernel for 'max_expansion = 5'
4. (Opt) Showing the performance of grf kernel with arbitrary modulation function. 

## Setup

In [1]:
import tensorflow as tf
import numpy as np
import scipy.special
import gpflow
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
import networkx as nx
from gpflow.utilities import print_summary
import tensorflow_probability as tfp
import seaborn as sns
from tqdm import tqdm

import sys
import os
project_root = os.path.abspath("../..")
sys.path.append(project_root)

from efficient_graph_gp.graph_kernels import get_normalized_laplacian
from efficient_graph_gp.gpflow_kernels import GraphDiffusionKernel
from utils import compute_fro
from traffic_utils.preprocessing import load_PEMS
from traffic_utils.plotting import plot_PEMS

2025-11-16 15:19:36.316868: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-16 15:19:36.329852: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1763306376.343573 2646675 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1763306376.348173 2646675 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1763306376.359093 2646675 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [2]:
os.chdir('../..')

In [3]:
!ls

algorithms	    efficient_graph_gp_sparse  presentations   requirements.txt
Archive		    experiments_dense	       processed_data  setup.py
data		    experiments_sparse	       __pycache__     utils.py
efficient_graph_gp  graph_bo		       README.md       venv


In [4]:
# Load and preprocess the PEMS dataset

np.random.seed(1111)
num_eigenpairs = 500
dataset = 'PeMS-Bay-new'
num_train = 250

G, data_train, data_test, data = load_PEMS(num_train=num_train)
x_train, y_train = data_train
x_test, y_test = data_test
x, y = data
orig_mean, orig_std = np.mean(y_train), np.std(y_train)
y_train = (y_train-orig_mean)/orig_std
y_test = (y_test-orig_mean)/orig_std

X_train = tf.convert_to_tensor(x_train)
X_full = tf.convert_to_tensor(x)
Y_train = tf.convert_to_tensor(y_train)

adjacency_matrix = nx.to_numpy_array(G)  # Converts to NumPy adjacency matrix
print(adjacency_matrix.shape)  # Check matrix size

/scratches/cartwright/mz473/Efficient-Gaussian-Process-on-Graphs/experiments_dense/traffic_dataset/traffic_utils/preprocessing.py:90: UserWarning: Unpickling a shapely <2.0 geometry object. Please save the pickle again as this compatibility may be removed in a future version of shapely.
  G = pickle.load(f)


epsg:4326


I0000 00:00:1763306407.809028 2646675 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9615 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:17:00.0, compute capability: 7.5
I0000 00:00:1763306407.810563 2646675 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 9611 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 2080 Ti, pci bus id: 0000:b3:00.0, compute capability: 7.5


(1016, 1016)


In [12]:
N = X_train.shape[0] # Number of training observations
M = 50  # Number of inducing locations

kernel = GraphDiffusionKernel(
    adjacency_matrix=adjacency_matrix
)
Z = X_train[:M, :].numpy().copy()  # Initialize inducing locations to the first M inputs in the dataset

m = gpflow.models.SVGP(kernel, gpflow.likelihoods.Gaussian(), Z, num_data=N)

In [13]:
def svgp_training(X, Y, model, num_iterations=300, learning_rate=0.01):
    """
    Train SVGP model using Adam optimizer
    
    Args:
        X: Training inputs
        Y: Training outputs
        model: SVGP model to train
        num_iterations: Number of optimization iterations
        learning_rate: Learning rate for Adam optimizer
    
    Returns:
        model: Trained SVGP model
        losses: List of loss values during training
    """
    optimizer = tf.optimizers.Adam(learning_rate=learning_rate)
    
    @tf.function
    def optimization_step():
        with tf.GradientTape() as tape:
            loss = model.training_loss((X, Y))
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        return loss
    
    losses = []
    for i in tqdm(range(num_iterations), desc='Training SVGP'):
        loss = optimization_step()
        losses.append(loss.numpy())
        
        if i % 100 == 0:
            print(f"Iteration {i}: Loss = {loss.numpy():.4f}")
    
    return model, losses

def svgp_evaluate(model, x_test, y_test, orig_std):
    """
    Evaluate SVGP model on test data
    
    Args:
        model: Trained SVGP model
        x_test: Test inputs
        y_test: Test outputs (standardized)
        orig_std: Original standard deviation for denormalization
    
    Returns:
        rmse: Root mean squared error
        nlpd: Negative log predictive density
    """
    # Get predictions (includes observation noise)
    mean_y, var_y = model.predict_y(x_test)
    
    # Reshape predictions
    mean_y = tf.reshape(mean_y, [-1])
    var_y = tf.reshape(var_y, [-1])
    
    # Compute RMSE (denormalized)
    rmse = float(orig_std * tf.sqrt(tf.reduce_mean((y_test[:, 0] - mean_y) ** 2)))
    
    # Compute NLPD
    nlpd = -tf.reduce_sum(
        tfp.distributions.Normal(loc=mean_y, scale=tf.sqrt(var_y)).log_prob(y_test[:, 0])
    ).numpy()
    
    return rmse, nlpd

In [14]:
# Train the SVGP model
trained_model, losses = svgp_training(X_train, Y_train, m, num_iterations=300, learning_rate=0.01)

Training SVGP:   0%|          | 1/300 [00:04<20:41,  4.15s/it]

Iteration 0: Loss = 397.3428


Training SVGP:  34%|███▎      | 101/300 [01:32<02:55,  1.13it/s]

Iteration 100: Loss = 363.4394


Training SVGP:  67%|██████▋   | 201/300 [03:00<01:27,  1.13it/s]

Iteration 200: Loss = 358.6187


Training SVGP: 100%|██████████| 300/300 [04:28<00:00,  1.12it/s]


In [15]:
print_summary(trained_model)

╒══════════════════════════╤═══════════╤══════════════════╤═════════╤═════════════╤═════════════╤═════════╤══════════════════════════════════════════════════════╕
│ name                     │ class     │ transform        │ prior   │ trainable   │ shape       │ dtype   │ value                                                │
╞══════════════════════════╪═══════════╪══════════════════╪═════════╪═════════════╪═════════════╪═════════╪══════════════════════════════════════════════════════╡
│ SVGP.kernel.beta         │ Parameter │ Softplus         │         │ True        │ ()          │ float64 │ 3.25914                                              │
├──────────────────────────┼───────────┼──────────────────┼─────────┼─────────────┼─────────────┼─────────┼──────────────────────────────────────────────────────┤
│ SVGP.kernel.sigma_f      │ Parameter │ Softplus         │         │ True        │ ()          │ float64 │ 0.3315358494831493                                   │
├─────────────────────

In [16]:
rmse, nlpd = svgp_evaluate(trained_model, x_test=x_test, y_test=y_test, orig_std=orig_std)
print(f"RMSE = {rmse}")
print(f"NLPD = {nlpd}")

RMSE = 16.402951419465193
NLPD = 102.91814576946457
